<a href="https://colab.research.google.com/github/thedarkhorse934/python-smart-contract-write-interaction/blob/main/python_smart_contract_write_interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.4 MB/s eta 0:00:00


In [8]:
from web3 import Web3

RPC_URL = "https://sepolia.infura.io/v3/YOUR_PROJECT_ID"   # or Alchemy etc
PRIVATE_KEY = "0xYOUR_PRIVATE_KEY"                         # your Sepolia-only burner
CONTRACT_ADDRESS = "0x334769cbC2E8cF92ef3dD9891e8D2Fd0578D09c5"

w3 = Web3(Web3.HTTPProvider(RPC_URL))
print("Connected:", w3.is_connected())

acct = w3.eth.account.from_key(PRIVATE_KEY)
MY_ADDRESS = acct.address
print("Your address:", MY_ADDRESS)

balance = w3.eth.get_balance(MY_ADDRESS)
print("Balance (ETH):", w3.from_wei(balance, "ether"))



Connected: True
Your address: 0x58E78A579f3FAc387fD8079F2db13C10307242Fc
Balance (ETH): 0.003898363165286744


In [9]:
COUNTER_ABI = [
    {
        "inputs": [],
        "name": "getCount",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    },
    {
        "inputs": [],
        "name": "increment",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function",
    },
]

In [10]:
contract = w3.eth.contract(
    address=Web3.to_checksum_address(CONTRACT_ADDRESS),
    abi=COUNTER_ABI
)

count_before = contract.functions.getCount().call()
print("Count before:", count_before)

Count before: 0


In [19]:
nonce = w3.eth.get_transaction_count(MY_ADDRESS)

tx = contract.functions.increment().build_transaction({
    "from": MY_ADDRESS,
    "nonce": nonce,
    "chainId": 11155111,  # Sepolia
})

# EIP-1559 fees (reasonable defaults for Sepolia)
tx["maxFeePerGas"] = w3.to_wei(30, "gwei")
tx["maxPriorityFeePerGas"] = w3.to_wei(2, "gwei")

# Estimate gas
tx["gas"] = w3.eth.estimate_gas(tx)

# Sign + send
signed = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)

print("Sent tx:", tx_hash.hex())
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Mined in block:", receipt.blockNumber)
print("Status:", receipt.status)


Sent tx: 7d9cf186474f24713cf8d2326e377f2a3ada2d7b7245d2b0ab01d1e444058ad5
Mined in block: 10104971
Status: 1


In [20]:
count_after = contract.functions.getCount().call()
print("Count after:", count_after)
print("Changed by:", count_after - count_before)

Count after: 1
Changed by: 1
